In [1]:
import import_ipynb
import torch
import dataloader
import generate_mahalanobis
import regression_mahalanobis
import odin
from models.resnet import ResNet34
from models.densenet import DenseNet3
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
id_datasets = ['cifar10', 'cifar100']
ood_dataset = 'svhn'
model_names = ['resnet34', 'densenet3']
batch_size = 128
magnitude = 0.0014

for id_dataset in id_datasets:
    if id_dataset == 'cifar10':
        num_classes = 10
    elif id_dataset == 'cifar100':
        num_classes = 100
        
    for model_name in model_names:
        if model_name == 'resnet34':
            model = ResNet34(num_c=num_classes).to(device)
        elif model_name == 'densenet3':
            model = DenseNet3(100, num_classes, growth_rate=12).to(device)
            
        model_path = f'./pretrained/{model_name}_{id_dataset}.pth'
        model.load_state_dict(torch.load(model_path, weights_only=True))
        model.eval()
            
        file_path = f'./output/{model_name}_{id_dataset}'
        
        if not os.path.exists(file_path):
            os.makedirs(file_path)
        
        mean, std = dataloader.get_mean_std(id_dataset)    
        
        id_trainloader, id_testloader = dataloader.get_imageloader(id_dataset, batch_size, mean, std)    
        _, out_testloader = dataloader.get_imageloader(ood_dataset, batch_size, mean, std)    
        
        generate_mahalanobis.mahalanobis(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path)
        regression_mahalanobis.regression(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')

num_samples: 10000
get sample mean and covariance

 Training Accuracy:(100.00%)

get Mahalanobis scores
20000
20000
[0. 0. 0. ... 1. 1. 1.]
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 86.60  97.23  92.33  97.58  96.00
Input noise: Mahalanobis_0.0014

num_samples: 10000
get sample mean and covariance

 Training Accuracy:(99.97%)

get Mahalanobis scores
20000
20000
[0. 0. 0. ... 1. 1. 1.]


c:\Users\Soyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 86.70  97.10  91.55  97.05  96.94
Input noise: Mahalanobis_0.0014



 13%|█▎        | 22.8M/169M [00:09<00:59, 2.47MB/s]


KeyboardInterrupt: 